# AG3 - Agente viajero
*Adrien Felipe*    
[Colab.research.google.com](https://colab.research.google.com/drive/1Dc8M-BSogYHAEieI95cRMWX0S7WAKAh_)    


In [1]:
!pip install tsplib95

In [0]:
import urllib.request
import tsplib95
import random
import math

In [0]:
file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)
problem = tsplib95.load_problem(file)

Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

# Busqueda aleatoria

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion

In [0]:
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

In [0]:
#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:
#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [0]:
#Busqueda aleatorio
def busqueda_aleatoria(problem, N):
  mejor_solucion = crear_solucion(Nodos)
  mejor_valor    = distancia_total(mejor_solucion, problem)

  for i in range(N):
      solucion = crear_solucion(Nodos)
      valor    = distancia_total(solucion, problem)
      if valor < mejor_valor:
        mejor_solucion = solucion
        mejor_valor = valor

  print("La mejor solucion es ",mejor_valor,  mejor_solucion)

In [9]:
busqueda_aleatoria(problem, 500)

La mejor solucion es  3813 [0, 16, 12, 41, 24, 5, 9, 23, 38, 30, 25, 8, 34, 20, 35, 17, 6, 19, 1, 13, 36, 15, 14, 18, 28, 37, 7, 32, 22, 39, 11, 33, 21, 31, 26, 27, 4, 3, 10, 40, 29, 2]


# Búsqueda local: Generador de vecindad

In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

In [0]:
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

In [12]:
busqueda_local(problem, 100)

La mejor solución encontrada es  1999 [0, 28, 29, 9, 23, 41, 8, 30, 34, 33, 20, 35, 36, 37, 7, 6, 26, 18, 39, 24, 40, 21, 5, 19, 13, 12, 11, 25, 10, 22, 38, 32, 31, 17, 1, 3, 27, 2, 4, 14, 16, 15]


# Recocido simulado

In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False

def bajar_temperatura(T):
  return T*.9999

In [15]:
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      #print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

La mejor solución encontrada es : 1478 [0, 37, 15, 16, 14, 19, 13, 18, 12, 11, 25, 10, 8, 28, 27, 2, 3, 4, 26, 5, 6, 1, 7, 32, 30, 29, 9, 41, 23, 40, 24, 21, 39, 22, 38, 34, 33, 20, 35, 36, 17, 31]


# Colonia de hormigas

In [0]:
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )

def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [0]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

In [18]:
hormigas(problem, 1000)

[0, 13, 15, 16, 14, 33, 19, 31, 36, 24, 37, 1, 11, 4, 2, 35, 12, 27, 30, 9, 40, 23, 10, 25, 26, 5, 18, 7, 29, 28, 38, 39, 22, 8, 6, 3, 21, 41, 34, 20, 17, 32]
3676


## Función de probabilidad  para elegir el siguiente nodo

In [0]:
def Add_Nodo(problem, camino, feromonas, peso_feromona=1, peso_distancia=1):
    """Función de probabilidad para elegir el siguiente nodo"""
    # Todos los nodos del problema.
    nodos = list(problem.get_nodes())
    # Ignoramos los nodos ya añadidos al camino.
    nodos_disponibles = list(set(range(1, len(nodos))) - set(camino))
    # Último nodo añadido al camino de la hormiga.
    nodo_actual = camino[-1]
    # Soporte para las probabilidades.
    probabilidades = []

    for nodo in nodos_disponibles:
        feromona = feromonas[nodo_actual][nodo]
        distancia_inversa = 1 / distancia(nodo_actual, nodo, problem)
        # No necesitamos respetar la formula y dividir cada probabilidad de forma que
        # suma(probabilidades) = 1. Obtendremos el mismo resultado.
        probabilidades.append(feromona ** peso_feromona * distancia_inversa ** peso_distancia)

    # Escogemos un punto aleatorio sobre el rango de probabilidades.
    seleccion = random.random() * sum(probabilidades)
    # Buscamos el nodo disponible correspondiente.
    rango = 0
    for posicion_nodo, probabilidad in enumerate(probabilidades):
        rango += probabilidad
        if seleccion < rango:
            break

    return nodos_disponibles[posicion_nodo]

In [20]:
hormigas(problem, 1000)

[0, 18, 26, 28, 1, 32, 19, 17, 12, 14, 16, 15, 37, 27, 2, 4, 41, 25, 23, 8, 29, 38, 30, 34, 20, 33, 31, 36, 35, 7, 3, 22, 39, 21, 40, 24, 6, 5, 13, 10, 9, 11]
2664
